data imports 

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator


pre trained model vgg-16

In [6]:
base_model_vg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model_vg.trainable = False 


adding more layers to pretrained model

In [7]:
model = Sequential([
    base_model_vg,
    Flatten(),
    Dense(128, activation='relu'),  
    BatchNormalization(),
    Dropout(0.7),   
    Dense(29, activation='softmax')   
])

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,930,333 (68.40 MB)

 Trainable params: 3,215,389 (12.27 MB)

 Non-trainable params: 14,714,944 (56.13 MB)

In [14]:
print(model.input_shape)


(None, 224, 224, 3)


the model optimizer


In [10]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

Data preparations


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=30,
    validation_split=0.2
    
)
# val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    r'C:\Users\Darsh\Desktop\Project\mp\asl_alphabet_train\asl_alphabet_train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    r'C:\Users\Darsh\Desktop\Project\mp\asl_alphabet_train\asl_alphabet_train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
    
)


Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


In [15]:
print(train_generator.class_indices)


{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28}


Model training 

In [17]:
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=val_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size
)


2175/2175 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2568 - loss: 2.7586

C:\Users\Darsh\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2175/2175 ━━━━━━━━━━━━━━━━━━━━ 5098s 2s/step - accuracy: 0.2569 - loss: 2.7584 - val_accuracy: 0.5608 - val_loss: 1.6936


In [19]:
# import matplotlib.pyplot as plt

# # Plot accuracy
# plt.plot(history.history['accuracy'], label='Train Accuracy')
# plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
# plt.legend()
# plt.title('Accuracy')
# plt.show()

# # Plot loss
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.legend()
# plt.title('Loss')
# plt.show()


Testing the data

In [ ]:
# # If you have a separate test dataset
# test_datagen = ImageDataGenerator(rescale=1./255)

# test_generator = test_datagen.flow_from_directory(
#     r'C:\Users\Darsh\Desktop\Project\mp\asl_alphabet_test\asl_alphabet_test',  
#     target_size=(224, 224),
#     batch_size=32,
#     class_mode='categorical'
# )

# # Evaluate the model
# test_loss, test_accuracy = model.evaluate(test_generator)
# print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
